In [1]:
import numpy as np
from importnb import Notebook
import heapq

with Notebook(): 
    import reproduccion_ulutas as rp



decoded antibodies [['P1', 'M1', 'M4'], ['M5'], ['M2', 'M3'], ['P6', 'P2', 'P4', 'P7', 'P3', 'P5']]
decoded antibodies [['P1', 'M2', 'P6', 'P3', 'P4', 'P2', 'P7', 'M3'], ['P5', 'M5', 'M1'], ['M4']]
decoded antibodies [['P1', 'M3', 'M5', 'M4'], ['M1'], ['P5'], ['P2', 'M2', 'P3', 'P7', 'P6', 'P4']]
[0.48571428571428577, 0.5609756097560975, 0.4871794871794872]
[0.0, 0.28048780487804875, 0.0]
[1 2]
decoded antibodies [['P1', 'M2', 'P6', 'P3', 'P4', 'P2'], ['M3'], ['P5', 'M5', 'M1'], ['M4', 'P7']]
decoded antibodies [['P1', 'M3', 'M5', 'M4'], ['M1'], ['P5', 'P7', 'P2', 'M2', 'P3'], ['P6', 'P4']]
[0.5263157894736842, 0.5000000000000001]
[0.2631578947368421, 0.0]
1 antibodies were updated
[0.48571428571428577, 0.5263157894736842, 0.4871794871794872]
[0.0, 0.2631578947368421, 0.0]
Se han borrado 2 anticuerpos
[[ 1  8 11  0  0 12  0  9 10  0  6  2  4  7  3  5]
 [ 1  9  6  3  4  2  7 10  0  5 12  8  0  0 11  0]]


In [2]:
POPULATION_SIZE = 3   # indicate the initial size of antibodies population
N = 0.5 # % of antibodies with highest affinities selected to be cloned
R = 0.5 # % of best cloned antibodies to the pool of antibodies
B = 0.5 # worst % of the whole population (RECEPTOR EDITING)
dataset = 'data/instances/testset_a/5x7_Waghodekar_Sahu(1984)[Problem-2].txt'
MAX_ITERATIONS = 1

log = True # Boolean: print logs at the execution of the algorithm.

## 1. Select N% of antibodies with highest affinities. 
        SELECTION ROULETTE WHEEL: antibodies that have higher efficacy values have a higher selection probability.
        N is a random number at each iteration
        
        Esos anticuerpos no se duplican, se clonan, se mutan y, si mejora su eficacia, son actualizados en el pool de anticuerpos

## 2. Incluir semilla en la **mutación** para que los resultados sean reproducibles
        

## 3. ~~Add new random antibodies into the population~~ 
~~La misma cantidad de anticuerpos que hemos borrado en el apartado anterior~~

## 4. Termination criteria
        Algorithm is terminated if no improvement is obtained in the efficacy value after 1000 iterations

## 5. ~~Penalties: calculation of affinity.~~
        each cell must contain at least one part and one machine. (cezali*(efficacy/2)) + (cresing*(efficacy/2))
        
        During calculation of the efficacy value, a penalty is defined because Cell#2 does not include any part. 
        Likewise, if a cell does not include any machine, a penalty value is defined and antibody’s objective function value gets a lower efficacy value. 
        The penalty value reduces the chance of the antibody to be selected in the next iteration
        

## 6. Mutation: if new efficacy value is higher, antibody is updated.
        Parece que la AFFINITY solo se utiliza en el primer paso del algoritmo, para hacer la selección del N% para la mutación

In [8]:
#reads data and translates it into part-machine matrix
matrix, m, p, columns, rows = rp.part_machine_incidence_matrix(dataset)
if log == True: print("\nPart-machine matrix\n",matrix)
#generation of initial pop.
antibodies = rp.generation_initial_population(p = p, m= m, population_size = POPULATION_SIZE, seed = 2021)
if log == True: print("\nAnticuerpos iniciales\n",antibodies)

iterations = 0
while iterations <= MAX_ITERATIONS:
    if log == True: print("\n Iteration number",iterations+1)
    #identifies cells in the antibodies
    total_cells = rp.cell_identification(antibodies = antibodies)
    if log == True: print("\n Cells\n", total_cells)

    #translates numbers into parts and machines
    total_machines, total_parts = rp.decode_cells(total_cells = total_cells) 

    #calculates part-machine matrix for each antibody
    antibody_matrices = rp.create_machine_part_matrix(matrix = matrix, 
                                                    antibodies = antibodies, 
                                                    total_machines = total_machines, 
                                                    total_parts = total_parts)
    #evaluates efficacie of each antibody
    efficacies, affinities, voids, exceptions = rp.evaluate_antibodies(antibody_matrices = antibody_matrices, total_cells = total_cells)
    if log == True: print("\n Efficacies",efficacies)
    if log == True: print("\n Affinities",affinities)

    # Select N% of antibodies with highest affinities. FALTA POR INCLUIR SELECTION ROULETTE WHEEL
    cloned_antibodies, positions_antibodies_selected = rp.antibodies_selection(antibodies=antibodies, N=N)
    if log == True: print("\n Cloned antibodies\n",cloned_antibodies)
    if log == True: print("\n Cloned antibodies positions\n",positions_antibodies_selected)
    # Mutation
    rp.mutate_cloned_antibodies(cloned_antibodies = cloned_antibodies)
    if log == True: print("\nAnticuerpos Clonados Mutados\n",cloned_antibodies)

    #CLONES: identifies cells in the antibody
    cloned_total_cells = rp.cell_identification(cloned_antibodies)
    #CLONES: translates numbers into parts and machines
    cloned_total_machines, cloned_total_parts = rp.decode_cells(cloned_total_cells)
    #CLONES: calculates part-machine matrix for each antibody
    cloned_antibody_matrices = rp.create_machine_part_matrix(matrix = matrix,
                                                                antibodies = cloned_antibodies, 
                                                                total_machines = cloned_total_machines, 
                                                                total_parts = cloned_total_parts)                                                          
    #CLONES evaluates efficacie of each antibody
    cloned_efficacies, cloned_affinities, cloned_voids, cloned_exceptions = rp.evaluate_antibodies(cloned_antibody_matrices,
                                                                                                    cloned_total_cells)
    if log == True: print("\n Cloned Efficacies",cloned_efficacies)
    if log == True: print("\n Cloned Affinities",cloned_affinities)
    # Add R% of best cloned antibodies to the pool of antibodies
    antibodies, efficacies, affinities = rp.select_best_cloned_antibodies(antibodies = antibodies,
                                        cloned_antibodies = cloned_antibodies,
                                        efficacies = efficacies, 
                                        cloned_efficacies = cloned_efficacies,
                                        affinities = affinities, 
                                        cloned_affinities = cloned_affinities,
                                        R = R,
                                        positions_antibodies_selected = positions_antibodies_selected)
    if log == True: print("\n New antibodies pool \n",antibodies)                                                                        
    # Remove worst members of the antibodies pool (receptor editing)
    antibodies, efficacies, affinities, amount_antibodies_erased = rp.receptor_editing(antibodies_pool = antibodies, 
                                                                                efficacies = efficacies, 
                                                                                affinities=affinities, 
                                                                                B = B)
    if log == True: print("\n Antibodies pool after removal \n", antibodies)
    #generate new random antibodies
    number_new_random_antibodies = amount_antibodies_erased
    new_random_antibodies = rp.generation_initial_population(p = p, 
                                                        m = m, 
                                                        population_size = number_new_random_antibodies, 
                                                        seed = 1995)
    if log == True: print("\n New random antibodies \n",new_random_antibodies)                                                        
    antibodies = np.concatenate((antibodies, new_random_antibodies), axis = 0)
    if log == True: print("\n Antibodies pool after adding new random antibodies \n",antibodies)
    #update iteration counter
    iterations = iterations + 1


#select best antibody. FALTA UNA EVALUACIÓN FINAL DE L



Part-machine matrix
     M1  M2  M3  M4  M5
P1   1   0   0   1   0
P2   0   1   0   1   1
P3   0   1   1   1   0
P4   0   1   1   1   1
P5   1   1   1   0   1
P6   1   0   1   0   1
P7   1   0   0   0   0

Anticuerpos iniciales
 [[ 4  0  0  3 11  2  8  0 12  9  0  7  1 10  6  5]
 [ 3  1  5  0  0  0  0 10  7 12  6  8  4  9  2 11]
 [ 0  4 11  5  6  1 10  9  8  7  2  0  3 12  0  0]]

 Iteration number 1

 Cells
 [[[4], [3, 11, 2, 8], [12, 9], [7, 1, 10, 6, 5]], [[3, 1, 5], [10, 7, 12, 6, 8, 4, 9, 2, 11]], [[4, 11, 5, 6, 1, 10, 9, 8, 7, 2], [3, 12]]]
decoded antibodies [['P4'], ['P3', 'M4', 'P2', 'M1'], ['M5', 'M2'], ['P7', 'P1', 'M3', 'P6', 'P5']]
decoded antibodies [['P3', 'P1', 'P5'], ['M3', 'P7', 'M5', 'P6', 'M1', 'P4', 'M2', 'P2', 'M4']]
decoded antibodies [['P4', 'M4', 'P5', 'P6', 'P1', 'M3', 'M2', 'M1', 'P7', 'P2'], ['P3', 'M5']]

 Efficacies [0.4871794871794872, 0.5909090909090909, 0.5957446808510638]

 Affinities [0.0, 0.29545454545454547, 0.5957446808510638]

 Cloned antibodies


In [ ]:
efficacies

array([0.59574468])